In [27]:
# messy imports to be tidied up later
import os
import xlutils, xypath
import databaker
import databaker.constants
from databaker.constants import *
import databaker.databakersolo as ds
from databaker.jupybakeutils import savepreviewhtml, savepreviewhtmlBAGS, writetechnicalCSV, procrows, TechnicalCSV, yield_dimension_values, DUPgenerate_header_row
Dcelllookup = ds.overrides.Dcelllookup
batchcelllookup = ds.overrides.batchcelllookup


In [28]:
"""
bake --preview ILCH.py ilchtablestemplatensa.xls growth
bake --preview ILCH.py ilchtablestemplatensa.xls level

bake --preview ILCH.py ilchtablestemplatesa.xls growth
bake --preview ILCH.py ilchtablestemplatesa.xls level
"""

# input-output
inputfile = "ilchtablestemplatesa.xls"
outputfile = "%s.csv" % os.path.splitext(inputfile)[0]
previewfile = "preview.html"


In [29]:
print("Loading %s which has size %d bytes" % (inputfile, os.path.getsize(inputfile)))
tableset = xypath.loader.table_set(inputfile, extension='xls')
tabs = list(xypath.loader.get_sheets(tableset, "*"))
print("Table names: %s" % ", ".join([tab.name  for tab in tabs]))


Loading ilchtablestemplatesa.xls which has size 407552 bytes
Table names: INTRODUCTION, DEFINITIONS, 1. Industry level SA, 2. Sector level SA, 3. Industry annual growth SA, 4. Sector annual growth SA, 5. Industry quarterly growth SA, 6. Sector quarterly growth SA


In [30]:
savepreviewhtmlBAGS(tabs[2], previewfile)

opening file preview.html
table '1. Industry level SA' written


In [31]:
newtabs = []
for tab in tabs:
    tab_title = tab.excel_ref("A1")
    if tab_title.filter(contains_string("growth")):    
        newtabs.append(tab)
tab = newtabs[0]
#[tab.name  for tab in newtabs]

In [34]:
conversionsegments = [ ]
for tab in tabs:
    tab_title = tab.excel_ref("A1")
    if not tab_title.filter(contains_string("growth")):    # this is the word that was in PARAMS(1)
        continue
        
    anchor = tab.filter("Period").assert_one()

    datarows = anchor.fill(DOWN).is_not_blank()
    datacols = anchor.shift(DOWN).fill(RIGHT).is_not_blank()
    
    # !!! this is where we are rewriting the values directly to strip out the bad names
    datarowsdim = HDim(datarows, TIME, DIRECTLY, LEFT)
    for c in datarows:
        datarowsdim.cellvalueoverride[c._cell] = c.value[:6]

    obs = datarows.waffle(datacols).is_not_blank()
    savepreviewhtmlBAGS(obs, previewfile)

    dimensions = [ 
        datarowsdim, 
        HDim(datacols.parent(), "Costs", DIRECTLY, ABOVE), 
        HDim(anchor.fill(RIGHT).parent(), "SIC", CLOSEST, LEFT)
    ]

    if tab_title.filter(contains_string("year on year")):
        dimensions.append(HDim("Annual", "Growth Period"))
        dimensions.append(HDim("Percent", MEASURETYPE))
    elif tab_title.filter(contains_string("quarter on quarter")):
        dimensions.append(HDim("Quarterly", "Growth Period"))
        dimensions.append(HDim("Percent", MEASURETYPE))
    elif tab_title.filter(contains_string("growth rates")):
        dimensions.append(HDim("Annual", "Growth Period"))
        dimensions.append(HDim("Percent", MEASURETYPE))
    else:
        assert 0 #tab.dimension(MEASURETYPE,"Index")
    if tab_title.filter(contains_string("Non-Seasonally")):
        dimensions.append(HDim("Not seasonally adjusted", "SA / NSA"))
    else:
        dimensions.append(HDim("Seasonally adjusted", "SA / NSA"))

    conversionsegment = (tab, dimensions, obs)
    conversionsegments.append(conversionsegment)
    
savepreviewhtml(conversionsegments[-1], previewfile)


opening file preview.html
table '3. Industry annual growth SA' written
opening file preview.html
table '4. Sector annual growth SA' written
opening file preview.html
table '5. Industry quarterly growth SA' written
opening file preview.html
table '6. Sector quarterly growth SA' written
opening file preview.html
table '6. Sector quarterly growth SA' written
javascript calculated


In [35]:
len(conversionsegments)

4

In [36]:
savepreviewhtml(conversionsegments[-1], previewfile)   # <---- preview thing

opening file preview.html
table '6. Sector quarterly growth SA' written
javascript calculated


In [38]:
print("converting and writing %d conversion segments into %s" % (len(conversionsegments), outputfile))
convertedrows = [ ]
for conversionsegment in conversionsegments:
    print("conversion segment size %d" % len(conversionsegment[2]))
    rows = procrows(conversionsegment)
    convertedrows.append(rows)
    
# Should detect errors and throw up preview with cells marked
# (though actually the errors manifest further upstream, normally)
    
# here we can do further sorting and postprocessing and allocating to different files 


converting and writing 4 conversion segments into ilchtablestemplatesa.csv
conversion segment size 5952
conversion segment size 2232
conversion segment size 6240
conversion segment size 2340


In [39]:

writetechnicalCSV(outputfile, conversionsegments)



writing 4 conversion segments into ilchtablestemplatesa.csv
conversionwrite segment size 5952
conversionwrite segment size 2232
conversionwrite segment size 6240
conversionwrite segment size 2340
